<a href="https://colab.research.google.com/github/ahcamachod/1904-word2vec-entrenamiento-de-word-embedding/blob/main/word2vec_entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Word2Vec: Entrenamiento de Word Embedding

En este notebook encontrarás el desarrollo del proyecto para modelar tus propias representaciones de Word Embedding utilizando **Word2Vec** .


Los principales recursos de Python que utilizaremos como base para nuestro modelaje se encuentran en:


*https://spacy.io/*

*https://radimrehurek.com/gensim/models/word2vec.html*


La documentación referente al diseño de las arquitecturas Word2Vec la encontramos en:


*https://arxiv.org/pdf/1301.3781.pdf*

## Aula 1

### 1.2 Iniciando con spacy

### 1.3 Spacy y sus estructuras

## Aula 2

### 2.1 Preprocesamiento de los datos

### 2.2 Optimizando el tratamiento de los datos

## Aula 3

### 3.1 Hiperparámetros de Word2Vec

### 3.2 Avanzando en los hiperparámetros

### 3.3 Vocabulario Word2Vec

## Aula 4

### 4.1 Entrenando el modelo CBOW

### 4.2 Entrenando el modelo Skip Gram

## Aula 5

### 5.1 Iniciando el clasificador

### 5.2 Combinación de vectores

### 5.3 Vectorizando los titulares

## Aula 6

### 6.1 Clasificando los titulares

### 6.2 Comparando los modelos